In [2]:
import os
import glob
import datetime as dt
import numpy as np
import xarray as xr
import pandas as pd
from sublimpy import utils, extrautils
import matplotlib.pyplot as plt
import altair as alt
alt.data_transformers.enable('json')
import act
import swifter

Open tidy dataset

In [3]:
sos_tidy_fn = f"../paper1/process_slow_data/tidy_df_20221101_20230619_planar_fit_multiplane_q7_flags360_pf10.parquet"
tidy_df = pd.read_parquet(sos_tidy_fn)
# Convert data timezone to local and clean up data on the ends
# convert time column to datetime
tidy_df['time'] = pd.to_datetime(tidy_df['time'])
tidy_df = utils.modify_df_timezone(tidy_df, 'UTC', 'US/Mountain')
# tidy_df = tidy_df[tidy_df.time > '20230301']
tidy_df = tidy_df[tidy_df.time > '20221130']
tidy_df = tidy_df[tidy_df.time < '20230509']
tidy_df.time.min(), tidy_df.time.max()

(Timestamp('2022-11-30 00:30:00'), Timestamp('2023-05-08 23:30:00'))

classify synoptic winds

In [4]:
USERNAME = os.getenv("ARM_USERNAME")
TOKEN = os.getenv("ARM_TOKEN")
SAIL_DATA_STREAM = 'gucdlprofwind4newsM1.c1'
DATA_STREAM_FILEEXT = '.nc'
startdate = "2022-11-30"
enddate = "2023-06-20"
output_dir = os.path.join("/Users/elischwat/Development/data/sublimationofsnow/", SAIL_DATA_STREAM)


In [5]:
# act.discovery.download_arm_data(USERNAME, TOKEN, SAIL_DATA_STREAM, startdate, enddate, output=output_dir)

In [6]:
dl_w_prof_files = glob.glob("/Users/elischwat/Development/data/sublimationofsnow/gucdlprofwind4newsM1.c1/*.nc")

In [7]:
prof_ds = act.io.read_arm_netcdf(dl_w_prof_files)
prof_ds.time.min(), prof_ds.time.max()

(<xarray.DataArray 'time' ()> Size: 8B
 array('2022-11-30T00:00:39.646250000', dtype='datetime64[ns]'),
 <xarray.DataArray 'time' ()> Size: 8B
 array('2023-06-15T23:59:20.789108000', dtype='datetime64[ns]'))

In [8]:
prof_df = prof_ds.to_dataframe()
prof_df = utils.modify_df_timezone(prof_df.reset_index(), 'UTC', 'US/Mountain').set_index('time')

In [9]:
hourly_mean_prof_df = prof_df.query("height > 950").query("height < 1100")[['wind_speed', 'wind_direction']].reset_index().set_index('time').groupby(pd.Grouper(freq='60min')).mean()

In [10]:
synoptic_winds_30min = hourly_mean_prof_df.resample('30min').mean().ffill()
along_valley_wind_times = synoptic_winds_30min[
    (synoptic_winds_30min.wind_direction > 299)
    & (synoptic_winds_30min.wind_direction < 345 )
].index.values
cross_valley_wind_times = synoptic_winds_30min[
    (synoptic_winds_30min.wind_direction > 202)
    & (synoptic_winds_30min.wind_direction < 248 )
].index.values

In [11]:
along_valley_wind_times = pd.DataFrame(along_valley_wind_times).set_index(0)
along_valley_wind_times.index.name = 'time'
along_valley_wind_times = along_valley_wind_times.assign(category = 'along-valley')
cross_valley_wind_times = pd.DataFrame(cross_valley_wind_times).set_index(0)
cross_valley_wind_times.index.name = 'time'
cross_valley_wind_times = cross_valley_wind_times.assign(category = 'cross-valley')

cross_valley_wind_times

,category
time,
2022-12-01 17:00:00,cross-valley
2022-12-01 17:30:00,cross-valley
2022-12-01 18:00:00,cross-valley
2022-12-01 18:30:00,cross-valley
2022-12-01 19:00:00,cross-valley
...,...
2023-06-15 10:30:00,cross-valley
2023-06-15 13:00:00,cross-valley
2023-06-15 13:30:00,cross-valley


In [12]:
synoptic_winds_pdf_data = pd.concat([
        synoptic_winds_30min[synoptic_winds_30min.index.isin(cross_valley_wind_times.index)].assign(synoptic_wind = 'cross valley'),
        synoptic_winds_30min[synoptic_winds_30min.index.isin(along_valley_wind_times.index)].assign(synoptic_wind = 'along valley')
    ])

In [13]:
fast_synoptic_wind_times = synoptic_winds_pdf_data.query("wind_speed <= 10").assign(synoptic_wind_speed = '<= 10 m/s')[['synoptic_wind_speed']]
slow_synoptic_wind_times = synoptic_winds_pdf_data.query("wind_speed > 10").assign(synoptic_wind_speed = '> 10 m/s')[['synoptic_wind_speed']]

Calculate scaling variables

In [14]:
src_variances = tidy_df[tidy_df.variable.isin([
    "w_w__3m_c",
    "u_u__3m_c",
    "v_v__3m_c",
    "u*_3m_c"
])].pivot_table(index='time', columns='variable', values='value')

src_variances['Phi_w'] = np.sqrt(src_variances['w_w__3m_c']) / src_variances['u*_3m_c']
src_variances['Phi_u'] = np.sqrt(src_variances['u_u__3m_c']) / src_variances['u*_3m_c']
src_variances['Phi_v'] = np.sqrt(src_variances['v_v__3m_c']) / src_variances['u*_3m_c']

In [15]:
k = 0.4
g = 9.81
src_lambda = tidy_df[tidy_df.variable.isin([
    "u*_3m_c",
    'Tpotvirtual_1m_c',
    'Tpotvirtual_2m_c',
    'Tpotvirtual_3m_c',
    'w_tc__3m_c',
    'SnowDepth_c'
])].pivot_table(index='time', columns='variable', values='value')
Lambda = - src_lambda['u*_3m_c']**3 * src_lambda[[
    'Tpotvirtual_1m_c',	'Tpotvirtual_2m_c',	'Tpotvirtual_3m_c', 
]].mean(axis=1) / (
    k*g*src_lambda['w_tc__3m_c']
)
z = 3 - src_lambda['SnowDepth_c']
z_over_Lambda = (z / Lambda).rename('z/L')

In [16]:
src_scaling = pd.DataFrame(z_over_Lambda).join(src_variances)

Categorize by synoptic wind direction and speed

In [17]:
src_scaling = pd.concat([
    src_scaling.join(along_valley_wind_times, how='inner'),
    src_scaling.join(cross_valley_wind_times, how='inner')
])

In [18]:
src_scaling = pd.concat([
    src_scaling.join(fast_synoptic_wind_times, how='inner'),
    src_scaling.join(slow_synoptic_wind_times, how='inner')
])


plot

In [19]:
src_scaling_stable = src_scaling[src_scaling['z/L'] > 0]
bins = pd.cut(src_scaling_stable['z/L'], np.logspace(-4,2,14))
src_scaling_stable['z/L bin'] = bins.apply(lambda bin: 0.5*(bin.left + bin.right)).astype(float)


src_scaling_unstable = src_scaling[src_scaling['z/L'] < 0]
bins = pd.cut(src_scaling_unstable['z/L'], ((-1)*np.logspace(-4,2,14))[::-1])
src_scaling_unstable['z/L bin'] = bins.apply(lambda bin: 0.5*(bin.left + bin.right)).astype(float)

/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_40494/2908346530.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  src_scaling_stable['z/L bin'] = bins.apply(lambda bin: 0.5*(bin.left + bin.right)).astype(float)
/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_40494/2908346530.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  src_scaling_unstable['z/L bin'] = bins.apply(lambda bin: 0.5*(bin.left + bin.right)).astype(float)


In [20]:
Phi_w_modeled_stable = lambda z_over_L: 1.25*(1 + 3*z_over_L)**(1/3)
Phi_w_modeled_unstable = lambda z_over_L: 1.25*(1 - 3*z_over_L)**(1/3)
Phi_w_model_src = pd.DataFrame({
    'model_domain_stable': np.logspace(-4, 1),
    'model_domain_unstable': -np.logspace(-4, 1),
    'model_range_stable': Phi_w_modeled_stable(np.logspace(-4, 1)),
    'model_range_unstable': Phi_w_modeled_unstable(-np.logspace(-4, 1)),
})

Phi_u_modeled_stable = lambda z_over_L: 2.55*(1 + 3*z_over_L)**(1/3)
Phi_u_modeled_unstable = lambda z_over_L: 2.55*(1 - 3*z_over_L)**(1/3)

Phi_u_model_src = pd.DataFrame({
    'model_domain_stable': np.logspace(-4, 1),
    'model_domain_unstable': -np.logspace(-4, 1),
    'model_range_stable': Phi_u_modeled_stable(np.logspace(-4, 1)),
    'model_range_unstable': Phi_u_modeled_unstable(-np.logspace(-4, 1)),
})

stable_model_phi_w_chart = alt.Chart(Phi_w_model_src).mark_line(color='black').encode(
        alt.X('model_domain_stable:Q').title(''),
        alt.Y('model_range_stable:Q').title(''),
    )
stable_model_phi_u_chart = alt.Chart(Phi_u_model_src).mark_line(color='black').encode(
        alt.X('model_domain_stable:Q').title(''),
        alt.Y('model_range_stable:Q').title(''),
    )
unstable_model_phi_w_chart = alt.Chart(Phi_w_model_src).mark_line(color='black').encode(
        alt.X('model_domain_unstable:Q').title(''),
        alt.Y('model_range_unstable:Q').title(''),
    )
unstable_model_phi_u_chart = alt.Chart(Phi_u_model_src).mark_line(color='black').encode(
        alt.X('model_domain_unstable:Q').title(''),
        alt.Y('model_range_unstable:Q').title(''),
    )

In [21]:
def chart(data):
    return alt.Chart(data).mark_point(size=50).encode(
        alt.X('z/L bin:Q').scale(type='log').title('z/L'),
        alt.Color('category:N'),
        alt.Shape('synoptic_wind_speed:N'),
    ).properties(width=200, height=150)
stable_base_chart = chart(
    src_scaling_stable[
        src_scaling_stable['z/L bin'] > 0.0002
    ][src_scaling_stable['z/L bin'] < 8]
)
unstable_base_chart = chart(
    src_scaling_unstable[
        src_scaling_unstable['z/L bin'] < -0.0002
    ][src_scaling_unstable['z/L bin'] > -8]
)
(
    (
        unstable_model_phi_w_chart + unstable_base_chart.encode(alt.Y('mean(Phi_w):Q').title('𝚽w')) 
        |
        stable_model_phi_w_chart + stable_base_chart.encode(alt.Y('mean(Phi_w):Q').title('𝚽w')) 
    ).resolve_scale(y='shared') & (
        (
            unstable_model_phi_u_chart + unstable_base_chart.encode(alt.Y('mean(Phi_u):Q').title('𝚽u'))
            |
            stable_model_phi_u_chart + stable_base_chart.encode(alt.Y('mean(Phi_u):Q').title('𝚽u'))
        ).resolve_scale(y='shared') & (
            unstable_model_phi_u_chart + unstable_base_chart.encode(alt.Y('mean(Phi_v):Q').title('𝚽v'))
            |
            stable_model_phi_u_chart + stable_base_chart.encode(alt.Y('mean(Phi_v):Q').title('𝚽v'))
        ).resolve_scale(y='shared')
    ).resolve_scale(y='shared')
).configure_axis(grid=False)

/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_40494/3777057509.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  src_scaling_stable[
/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_40494/3777057509.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  src_scaling_unstable[


alt.VConcatChart(...)